In [37]:
### 1. import data
 
from mnist import MNIST
 
data = MNIST(path='data/', return_type='numpy')
data.select_emnist('letters')
X, y = data.load_training()
 
print(X.shape, y.shape) # 28*28

(124800, 784) (124800,)


In [38]:
X = X.reshape(124800, 28, 28)
y = y.reshape(124800, 1)

In [39]:
# list(y) #--> y ranges from 1 to 26

In [40]:
y = y-1

In [41]:
 #list(y) #--> y ranges from 1 to 25

In [42]:
### 2. train-test split
 
# pip install scikit-learn
from sklearn.model_selection import train_test_split

In [43]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=50)

In [44]:
# (0,255) --> (0,1)
X_train = X_train.astype('float32')/255
X_test = X_test.astype('float32')/255

In [45]:
# y_train, y_test

In [46]:
import tensorflow as tf

# integer into one hot vector (binary class matrix)
y_train = tf.keras.utils.to_categorical(y_train, num_classes=26)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=26)


In [47]:
#y_train, y_test

In [48]:
### 3. Define our model
 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten

model = Sequential()
model.add(Flatten(input_shape=(28, 28)))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))  # Preventing overfitting
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(26, activation='softmax'))
 

In [49]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_3 (Flatten)         (None, 784)               0         
                                                                 
 dense_9 (Dense)             (None, 512)               401920    
                                                                 
 dropout_6 (Dropout)         (None, 512)               0         
                                                                 
 dense_10 (Dense)            (None, 512)               262656    
                                                                 
 dropout_7 (Dropout)         (None, 512)               0         
                                                                 
 dense_11 (Dense)            (None, 26)                13338     
                                                                 
Total params: 677914 (2.59 MB)
Trainable params: 67791

In [50]:
from tensorflow.keras.optimizers import Adam

model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])


In [51]:
### 4. calculate accuracy
 
score = model.evaluate(X_test, y_test, verbose=0)
accuracy = 100 * score[1]
print("Before training, test accuracy is", accuracy)

Before training, test accuracy is 4.767628386616707


In [53]:
checkpointer = ModelCheckpoint(filepath='best_model', verbose=1, save_best_only=True)

# Train the model
history = model.fit(X_train, y_train, batch_size=128, epochs=10, 
                    validation_split=0.2, callbacks=[checkpointer], verbose=1, shuffle=True)

# After training, load the best model
model = tf.keras.models.load_model('best_model')

# Calculate accuracy after training
score = model.evaluate(X_test, y_test, verbose=0)
accuracy = 100 * score[1]
print("After training, test accuracy is", accuracy)

Epoch 1/10
623/624 [============================>.] - ETA: 0s - loss: 0.1859 - accuracy: 0.9330
Epoch 1: val_loss improved from inf to 0.28391, saving model to best_model
INFO:tensorflow:Assets written to: best_model\assets


INFO:tensorflow:Assets written to: best_model\assets


624/624 [==============================] - 23s 35ms/step - loss: 0.1859 - accuracy: 0.9330 - val_loss: 0.2839 - val_accuracy: 0.9163
Epoch 2/10
623/624 [============================>.] - ETA: 0s - loss: 0.1730 - accuracy: 0.9365
Epoch 2: val_loss did not improve from 0.28391
624/624 [==============================] - 20s 32ms/step - loss: 0.1729 - accuracy: 0.9366 - val_loss: 0.2864 - val_accuracy: 0.9135
Epoch 3/10
624/624 [==============================] - ETA: 0s - loss: 0.1637 - accuracy: 0.9386
Epoch 3: val_loss did not improve from 0.28391
624/624 [==============================] - 20s 31ms/step - loss: 0.1637 - accuracy: 0.9386 - val_loss: 0.3011 - val_accuracy: 0.9132
Epoch 4/10
623/624 [============================>.] - ETA: 0s - loss: 0.1550 - accuracy: 0.9422
Epoch 4: val_loss did not improve from 0.28391
624/624 [==============================] - 19s 30ms/step - loss: 0.1552 - accuracy: 0.9421 - val_loss: 0.2977 - val_accuracy: 0.9158
Epoch 5/10
624/624 [===================

In [4]:

import tensorflow as tf
 
model = tf.keras.models.load_model('best_model')
 
letters ={ 0:'a', 1:'b', 2:'c', 3:'d', 4:'e', 5:'f', 6:'g', 7:'h', 8:'i', 9:'j', 10:'k', 11:'l', 
          12:'m', 13:'n', 14:'o', 15:'p', 16:'q', 17:'r', 18:'s', 19:'t', 20:'u', 21:'v', 22:'w', 
          23:'x', 24:'y', 25:'z', 26:''}
 
# defining blue color in hsv format
# pip install numpy
import numpy as np
 
blueLower = np.array([100,60,60])
blueUpper = np.array([140,255,255])
 
kernel = np.ones((5,5), np.uint8)
 
# define blackboard
blackboard = np.zeros((480,640, 3), dtype=np.uint8)
alphabet = np.zeros((200,200,3), dtype=np.uint8)
 
# deques (Double ended queue) is used to store alphabet drawn on screen
from collections import deque
points = deque(maxlen = 512)

In [5]:
import cv2 #pip install opencv-python
cap = cv2.VideoCapture(0)
prediction = 26
while True:
    ret, frame=cap.read()
    frame = cv2.flip(frame, 1)
     
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
     
    # Detecting which pixel value falls under blue color boundaries
    blue = cv2.inRange(hsv, blueLower, blueUpper)
     
    #erosion
    blue = cv2.erode(blue, kernel)
    #opening
    blue = cv2.morphologyEx(blue, cv2.MORPH_OPEN, kernel)
    #dilation
    blue = cv2.dilate(blue, kernel)
     
    # find countours in the image
    cnts , _ = cv2.findContours(blue, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
      
    center = None
     
    # if any countours were found
    if len(cnts) > 0:
        cnt = sorted(cnts, key = cv2.contourArea, reverse=True)[0]
        ((x,y), radius) = cv2.minEnclosingCircle(cnt)
        cv2.circle(frame, (int(x), int(y),), int(radius), (125,344,278), 2)
         
        M = cv2.moments(cnt)
        center = (int(M['m10']/M['m00']), int(M['m01']/M['m00']))
     
        points.appendleft(center)
         
    elif len(cnts) == 0:
        if len(points) != 0:
            blackboard_gray = cv2.cvtColor(blackboard, cv2.COLOR_BGR2GRAY)
            blur = cv2.medianBlur(blackboard_gray, 15)
            blur = cv2.GaussianBlur(blur, (5,5), 0)
            thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)[1]
            cv2.imshow("Thresh", thresh)
             
            blackboard_cnts = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)[0]
             
            if len(blackboard_cnts)>=1:
                cnt = sorted(blackboard_cnts, key=cv2.contourArea, reverse=True)[0]
                 
                if cv2.contourArea(cnt)>1000:
                    x,y,w,h = cv2.boundingRect(cnt)
                    alphabet = blackboard_gray[y-10:y+h+10,x-10:x+w+10]
                    try:
                        img = cv2.resize(alphabet, (28,28))
                    except cv2.error as e:
                        continue
                     
                    img = np.array(img)
                    img = img.astype('float32')/255
                     
                    prediction = model.predict(img.reshape(1,28,28))[0]
                    prediction = np.argmax(prediction)
                     
            # Empty the point deque and also blackboard
            points = deque(maxlen=512)
            blackboard = np.zeros((480,640, 3), dtype=np.uint8)
         
    # connect the detected points with line
    for i in range(1, len(points)):
        if points[i-1] is None or points[i] is None:
            continue
        cv2.line(frame, points[i-1], points[i], (0,0,0), 2)
        cv2.line(blackboard, points[i-1], points[i], (255,255,255), 8)
         
     
    cv2.putText(frame, "Prediction: " + str(letters[int(prediction)]), (20,400), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2)
     
     
    cv2.imshow("Alphabet Recognition System", frame)
     
    if cv2.waitKey(1)==13: #if I press enter
        break
cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 111ms/step


In [2]:
import cv2
import numpy as np
from collections import deque

import tensorflow as tf

# Define color boundaries for blue
blueLower = np.array([100, 60, 60])
blueUpper = np.array([140, 255, 255])

# Create kernel for morphological transformations
kernel = np.ones((5, 5), np.uint8)

# Initialize deque to store points
points = deque(maxlen=512)

# Initialize blackboard to draw on
blackboard = np.zeros((480, 640, 3), dtype=np.uint8)

# Load your model
model = tf.keras.models.load_model('best_model')  # Replace 'your_model_path.h5' with your model path

# Define letters (adjust this as per your model's output)
letters = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"

# Initialize a list to store recognized alphabets
recognized_alphabets = []

cap = cv2.VideoCapture(0)
prediction = 26

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    frame = cv2.flip(frame, 1)
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
     
    # Detecting which pixel value falls under blue color boundaries
    blue = cv2.inRange(hsv, blueLower, blueUpper)
     
    # Erosion, opening, and dilation
    blue = cv2.erode(blue, kernel)
    blue = cv2.morphologyEx(blue, cv2.MORPH_OPEN, kernel)
    blue = cv2.dilate(blue, kernel)
     
    # Find contours in the image
    cnts, _ = cv2.findContours(blue, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
      
    center = None
     
    # If any contours were found
    if len(cnts) > 0:
        cnt = sorted(cnts, key=cv2.contourArea, reverse=True)[0]
        ((x, y), radius) = cv2.minEnclosingCircle(cnt)
        cv2.circle(frame, (int(x), int(y)), int(radius), (125, 344, 278), 2)
         
        M = cv2.moments(cnt)
        center = (int(M['m10'] / M['m00']), int(M['m01'] / M['m00']))
     
        points.appendleft(center)
         
    elif len(cnts) == 0:
        if len(points) != 0:
            blackboard_gray = cv2.cvtColor(blackboard, cv2.COLOR_BGR2GRAY)
            blur = cv2.medianBlur(blackboard_gray, 15)
            blur = cv2.GaussianBlur(blur, (5, 5), 0)
            thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
            cv2.imshow("Thresh", thresh)
             
            blackboard_cnts, _ = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
             
            if len(blackboard_cnts) >= 1:
                cnt = sorted(blackboard_cnts, key=cv2.contourArea, reverse=True)[0]
                 
                if cv2.contourArea(cnt) > 1000:
                    x, y, w, h = cv2.boundingRect(cnt)
                    alphabet = blackboard_gray[y-10:y+h+10, x-10:x+w+10]
                    try:
                        img = cv2.resize(alphabet, (28, 28))
                    except cv2.error as e:
                        continue
                     
                    img = np.array(img)
                    img = img.astype('float32') / 255
                     
                    prediction = model.predict(img.reshape(1, 28, 28))[0]
                    prediction = np.argmax(prediction)
                    
                    # Append the recognized alphabet to the list
                    recognized_alphabets.append(letters[prediction])
                     
            # Empty the point deque and also blackboard
            points = deque(maxlen=512)
            blackboard = np.zeros((480, 640, 3), dtype=np.uint8)
         
    # Connect the detected points with a line
    for i in range(1, len(points)):
        if points[i-1] is None or points[i] is None:
            continue
        cv2.line(frame, points[i-1], points[i], (0, 0, 0), 2)
        cv2.line(blackboard, points[i-1], points[i], (255, 255, 255), 8)
         
    # Ensure the prediction is within the valid range
    if 0 <= prediction < len(letters):
        cv2.putText(frame, "Prediction: " + str(letters[prediction]), (20, 400), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
     
    cv2.imshow("Alphabet Recognition System", frame)
     
    if cv2.waitKey(1) == 13:  # if I press enter
        break

cap.release()
cv2.destroyAllWindows()

# Join the recognized alphabets into a single string and print it
recognized_string = ''.join(recognized_alphabets)
print("Recognized Alphabets:", recognized_string)


1/1 [==============================] - 0s 40ms/step
Recognized Alphabets: GAKBCBBDCCEFGHIJKKLMNOPRQKRBBKSPTWUVWXYDWXZ
